# Data import

In [1]:
import sys

sys.path.insert(1, 'Cluster/')

from data_load import *
    

In [2]:
data = import_Gaia_data("data/type2.csv")

import matplotlib.pyplot as plt
import numpy as np

plt.clf()
plt.scatter (np.sin (data.positions[:,1]) , data.proper_motions_err[:,0]/numpy.cos(data.positions[:,1]))
plt.savefig("bug.png")

# Data conversion and coordinates change

In [4]:
# Change positions from deg to rad

data.positions = deg_to_rad (data.positions)

In [5]:
# Change proper motions from mas/yr to rad/s

data.proper_motions = data.proper_motions * 1.5362818500441604e-16
data.proper_motions_err = data.proper_motions_err * 1.5362818500441604e-16

In [9]:
# Calculate covariance matrix and its inverse

data.covariance = covariant_matrix ( data.proper_motions_err , data.proper_motions_err_corr )

data.covariance_inv = numpy.linalg.inv ( data.covariance )

In [20]:
# Calculate Cartesian positions
import sys

sys.path.insert(1, 'Cluster/')

from CoordinateTransformations import *

data.positions_Cartesian = geographic_to_Cartesian_point ( data.positions )

In [15]:
# Change proper motions from (ra,dec) to Cartesian coordinates

#data3.proper_motions = tangent_geographic_to_Cartesian2 (data3.positions , data3.proper_motions)
#data3.proper_motions_err = tangent_geographic_to_Cartesian2 (data3.positions , data3.proper_motions_err)

In [16]:
# Change positions from (ra,dec) to Cartesian

#data3.positions = geographic_to_Cartesian (data3.positions)

# Generating random VSH coefficients

In [17]:
import random
from VectorSphericalHarmonicsVectorized import VectorSphericalHarmonicE, VectorSphericalHarmonicB

def random_aQlm_coeffs ( lmax , lower_bound , upper_bound ):
    negative_coeffs = [ [ random.uniform ( lower_bound , upper_bound ) + 1j * random.uniform ( lower_bound , upper_bound ) for m in range ( -l , 0 ) ] for l in range ( 1 , lmax + 1 ) ]
    
    coeffs = [ [ negative_coeffs[ l-1 ][ m+l ] if m < 0
                 else random.uniform ( lower_bound , upper_bound ) + 1j * 0.0 if m == 0
                 else (-1) ** m * numpy.conj ( negative_coeffs[ l-1 ][ m-l ] )
                 for m in range ( -l , l+1 ) ] for l in range ( 1 , lmax + 1 ) ]
    
    return coeffs

def random_vsh_coeffs ( lmax , lower_bound , upper_bound):
    vsh_E_coeffs = random_aQlm_coeffs ( lmax , lower_bound , upper_bound )
    vsh_B_coeffs = random_aQlm_coeffs ( lmax , lower_bound , upper_bound )
        
    return vsh_E_coeffs , vsh_B_coeffs


In [18]:
vsh_E_coeffs , vsh_B_coeffs = random_vsh_coeffs ( 2 , -1.0e-15 , 1.0e-15)

# Generate data model

In [59]:
from CoordinateTransformations import *

Lmax = 1

par = {}

for l in numpy.arange(1,Lmax+1):
    for m in numpy.arange(0, l+1):
        if m==0:
            par['Re_a^E_'+str(l)+'0'] = 0
            par['Re_a^B_'+str(l)+'0'] = 0
        else:
            par['Re_a^E_'+str(l)+str(m)] = 0
            par['Im_a^E_'+str(l)+str(m)] = 0
            par['Re_a^B_'+str(l)+str(m)] = 0
            par['Im_a^B_'+str(l)+str(m)] = 0
            
VSH_bank = {}

for l in range ( 1 , Lmax + 1 ):
    VSH_bank['Re[Y^E_' + str(l) + '0]'] = Cartesian_to_geographic_vector (data.positions_Cartesian , numpy.real ( VectorSphericalHarmonicE ( l , 0 , data.positions_Cartesian ) ) )
        
    VSH_bank['Re[Y^B_' + str(l) + '0]'] = Cartesian_to_geographic_vector (data.positions_Cartesian , numpy.real ( VectorSphericalHarmonicB ( l , 0 , data.positions_Cartesian ) ) )
    
    for m in range ( 1 , l + 1 ):
        VSH_bank['Re[Y^E_' + str(l) + str(m) + ']'] = Cartesian_to_geographic_vector (data.positions_Cartesian , numpy.real ( VectorSphericalHarmonicE ( l , m , data.positions_Cartesian ) ) )
        
        VSH_bank['Im[Y^E_' + str(l) + str(m) + ']'] = Cartesian_to_geographic_vector (data.positions_Cartesian , numpy.imag ( VectorSphericalHarmonicE ( l , m , data.positions_Cartesian ) ) )
        
        VSH_bank['Re[Y^B_' + str(l) + str(m) + ']'] = Cartesian_to_geographic_vector (data.positions_Cartesian , numpy.real ( VectorSphericalHarmonicB ( l , m , data.positions_Cartesian ) ) )
        
        VSH_bank['Im[Y^B_' + str(l) + str(m) + ']'] = Cartesian_to_geographic_vector (data.positions_Cartesian , numpy.imag ( VectorSphericalHarmonicB ( l , m , data.positions_Cartesian ) ) )

def generate_model ( coeffs , VSH_bank ):
    v_Q = numpy.sum ( [ numpy.sum ( [ 
                        coeffs['Re_a^' + Q + '_' + str(l) + '0'] * VSH_bank['Re[Y^' + Q + '_' + str(l) + '0]'] 
                        + 2 * numpy.sum ( [ 
                        coeffs['Re_a^' + Q + '_'+str(l)+str(m)] * VSH_bank['Re[Y^' + Q + '_' + str(l) + str(m) + ']'] 
                        - coeffs['Im_a^'+ Q + '_'+str(l)+str(m)] * VSH_bank['Im[Y^' + Q + '_' + str(l) + str(m) + ']'] 
                        for m in range ( 1 , l + 1 ) ] )
                    for l in range ( 1 , Lmax + 1 ) ] )
                for Q in [ 'E' , 'B' ] ] )
        
    return v_Q


generate_model ( par , VSH_bank )

0.0

In [97]:
model = generate_model ( vsh_E_coeffs , vsh_B_coeffs , data3.positions )

# Compute the log-likelihood

In [8]:
def covariant_matrix ( errors , corr ):
    covariant_matrix = numpy.einsum ( '...i,...j->...ij' , errors , errors )
    
    covariant_matrix[...,0,1] = covariant_matrix[...,1,0] = numpy.multiply ( covariant_matrix[...,1,0] , corr.flatten() )
    return covariant_matrix

def R_values ( pm , pm_err , pm_err_corr , model ):
    covariant_matrices = covariant_matrix ( pm_err , pm_err_corr )
    
    M = pm - model
    
    R_values = numpy.einsum ( '...i,...ij,...j->...' , M , numpy.linalg.inv ( covariant_matrices ) , M ) 
        
    return R_values

def compute_log_likelihood ( R_values ):
    log_likelihood = numpy.log ( ( 1. - numpy.exp ( - R_values ** 2 / 2.) ) / ( R_values ** 2 ) ).sum()
    
    return log_likelihood


In [99]:
R = R_values ( data3.proper_motions , data3.proper_motions_err , data3.proper_motions_err_corr , model)

u = compute_log_likelihood (R)

u

-27058.36292655995

In [3]:
points_Cart = geographic_to_Cartesian_point ( data.positions )

from VectorSphericalHarmonicsVectorized import VectorSphericalHarmonicE, VectorSphericalHarmonicB

n_objects = points_Cart.shape[0]

prefactor = 4 * numpy.pi / n_objects

import numpy as np
names = []
F = {}


Lmax = 2

l_range = numpy.arange(1,Lmax+1)

for Q in [ 'E' , 'B' ]:
    for l in np.arange(1, Lmax+1):
        for m in np.arange(0, l+1):
            if m == 0:
                names += [ 'Re[a^' + Q + '_' + str(l) + str(m) + ']' ]

            else:
                names += [ 'Re[a^' + Q + '_' + str(l) + str(m) + ']' ]
                names += [ 'Im[a^' + Q + '_' + str(l) + str(m) + ']' ]
                
matrix = np.zeros( (len(names), len(names)) )        
        
for i, n_x in enumerate(names):
    part_x = n_x.split('^')[0][0]
    Q_x = n_x.split('^')[1][0]
    l_x = int(n_x.split('^')[1][2])
    m_x = int(n_x.split('^')[1][3])
    
    for j, n_y in enumerate(names):
        part_y = n_y.split('^')[0][0]
        Q_y = n_y.split('^')[1][0]
        l_y = int(n_y.split('^')[1][2])
        m_y = int(n_y.split('^')[1][3])
        
        X = VectorSphericalHarmonicE ( l_x , m_x , points_Cart ) if Q_x=='E' else VectorSphericalHarmonicB ( l_x , m_x , points_Cart )
        X = np.real(X) if part_x=='R' else np.imag(X)
        
        Y = VectorSphericalHarmonicE ( l_y , m_y , points_Cart ) if Q_y=='E' else VectorSphericalHarmonicB ( l_y , m_y , points_Cart )
        Y = np.real(Y) if part_y=='R' else np.imag(Y)
        
        matrix[i,j] = prefactor * numpy.einsum ( "...j,...j->..." , X , Y ).sum()
        
plt.clf()
plt.imshow(matrix)
plt.colorbar()
plt.savefig("matrix.pdf")
plt.clf()

corr = []
for i in range(len(matrix)):
    for j in range(i):
        x = matrix[i,j]/np.sqrt(matrix[i,i]*matrix[j,j])
        corr.append(x)
        if abs(x)>0.14:
            print(names[i], names[j])
corr = np.array(corr)
plt.hist(corr)
plt.savefig("hist.png")

i = names.index('Im[a^E_11]')
j = names.index('Re[a^B_21]')
x = matrix[i,j]/np.sqrt(matrix[i,i]*matrix[j,j])
print (x)

Re[a^E_21] Re[a^E_11]
Im[a^E_21] Im[a^E_11]
Re[a^B_11] Im[a^E_11]
Im[a^B_11] Re[a^E_11]
Im[a^B_11] Re[a^E_21]
Re[a^B_21] Im[a^E_11]
Re[a^B_21] Re[a^B_11]
Im[a^B_21] Im[a^B_11]
-0.18013151269390593


# Trying to fit with Chris' Python Particle Swarm Optimisation

In [14]:
import PySO

class QSO_VSH_fit(PySO.Model):

    Lmax = 3
    
    names=[]
    for L in numpy.arange(1, Lmax+1):
        for m in numpy.arange(-L, L+1):
            if m==0:
                names += ['Ra^E_{0}_{1}'.format(L,m)]
                names += ['Ra^B_{0}_{1}'.format(L,m)]
            else:
                names += ['Ra^E_{0}_{1}'.format(L,m)]
                names += ['Ra^B_{0}_{1}'.format(L,m)]
                names += ['Ia^E_{0}_{1}'.format(L,m)]
                names += ['Ia^B_{0}_{1}'.format(L,m)]
    
    bounds = [ [-1.0e-14,1.0e-14] for i in range(len(names))]
    

    def log_likelihood(self, param):
        
        vsh_E_coeffs = [ [ 
                            param['Ra^E_{0}_{1}'.format(L,m)] 
                            if m==0 else 
                            param['Ra^E_{0}_{1}'.format(L,m)]+(1j)*param['Ia^E_{0}_{1}'.format(L,m)]
                          for m in numpy.arange(-L, L+1) ] for L in numpy.arange(1, self.Lmax+1)]
        
        vsh_B_coeffs = [ [ 
                            param['Ra^B_{0}_{1}'.format(L,m)] 
                            if m==0 else 
                            param['Ra^B_{0}_{1}'.format(L,m)]+(1j)*param['Ia^B_{0}_{1}'.format(L,m)]
                          for m in numpy.arange(-L, L+1) ] for L in numpy.arange(1, self.Lmax+1)]
        
        print (vsh_E_coeffs)
        
        model = generate_model (vsh_E_coeffs , vsh_B_coeffs , data3.positions)
        R = generate_R ( data3.proper_motions , data3.proper_motions_err , data3.proper_motions_err_corr , model)
        return log_likelhood_fun (R)
    
mymodel = QSO_VSH_fit()

NumParticles = 2

myswarm = PySO.Swarm(mymodel,
                NumParticles,
                Omega = 0.01,
                PhiP = 0.1,
                PhiG = 0.1,
                EnergyTol = 1.0e-8)

myswarm.Run()

[[(-4.051820432552491e-15+3.9019242260068935e-15j), 1.0962514403063393e-15, (-2.1989423583619877e-15+1.5898338470668441e-15j)], [(-4.005078512411002e-15+8.295448331349153e-15j), (7.242640554933391e-15-7.022912072523901e-15j), 7.407801139573162e-16, (-5.394325112551346e-15+9.140480362437914e-15j), (-3.866305737185125e-15-1.4372516862603125e-15j)], [(9.905129645633751e-15-1.2045374537853373e-15j), (-4.591105300268807e-15+1.6206152525940413e-16j), (-1.4651795083578574e-15-2.3169299929584544e-15j), 2.00678369665432e-15, (8.961771647529057e-15-3.435506878025172e-15j), (-2.777910391239437e-15-6.516987950664284e-15j), (-7.6636705852496e-15+1.4687411361732721e-15j)]]
[[(-4.86130129408709e-15+9.746539081626012e-15j), 1.7986352892125863e-15, (7.270009629945627e-15-2.0299349018935748e-15j)], [(-3.490502032823688e-15-1.715352162705143e-15j), (-4.862114468593031e-15-5.833711376264423e-15j), -8.561320209723371e-15, (4.63599678108349e-15+5.5491893756491685e-15j), (-8.754921473469426e-15+7.54194650712

KeyboardInterrupt: 

# Experiments

In [6]:
import timeit

timeit.timeit('"-".join(str(n) for n in range(100))', number=100)

0.0029944289999548346

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.patches import Ellipse

def plot_pm_err_ellipse (pm , pm_err , pm_err_corr):
    i = 575
    
    covariance_matrix = numpy.array ( [ [ pm_err[i,0] * pm_err[i,0] , pm_err_corr[i] * pm_err[i,0] * pm_err[i,1] ],
                                        [ pm_err_corr[i] * pm_err[i,1] * pm_err[i,0] , pm_err[i,1] * pm_err[i,1] ] ] )
    
    eigenvalues , eigenvectors = numpy.linalg.eig ( covariance_matrix )
    
    box_size = numpy.max ( [ numpy.sqrt(eigenvalues[0]) , numpy.sqrt(eigenvalues[1]) , numpy.linalg.norm ( pm[i] ) ] )
    
    plt.figure(figsize=(10,10))
    plt.axes().set_aspect('equal')
    ax = plt.gca()
    
    plt.xlim ( [ pm[i,0] - 1.5 * box_size , pm[i,0] + 1.5 * box_size ] )
    plt.ylim ( [ pm[i,1] - 1.5 * box_size , pm[i,1] + 1.5 * box_size ] )
    
    plt.arrow(0 , 0 , pm[i,0] , pm[i,1] , width=box_size * 1e-3 , head_width=box_size * 0.5e-1, head_length=box_size * 1e-1 , color='k')
            
    plt.arrow(pm[i,0] , pm[i,1] , numpy.sqrt(eigenvalues[0]) * eigenvectors[0,0] , numpy.sqrt(eigenvalues[0]) * eigenvectors[0,1] , width=box_size * 1e-3 , head_width=box_size * 0.5e-1, head_length=box_size * 1e-1 , color='orange')
    
    plt.arrow(pm[i,0] , pm[i,1] , numpy.sqrt(eigenvalues[1]) * eigenvectors[1,0] , numpy.sqrt(eigenvalues[1]) * eigenvectors[1,1] , width=box_size * 1e-3 , head_width=box_size * 0.5e-1, head_length=box_size * 1e-1 , color='y')
    
    angle = - numpy.rad2deg ( numpy.arctan2 ( eigenvectors[1,0] , eigenvectors[1,1] ) )
            
    ell = Ellipse(xy=(pm[i,0] , pm[i,1]), width=2 * numpy.sqrt(eigenvalues[0]), height=2 * numpy.sqrt(eigenvalues[1]), angle = angle)
    
    ax.add_patch(ell)
    ax.set_aspect('equal')
            
    plt.show()
    
    return covariance_matrix

print (plot_pm_err_ellipse ( data3.proper_motions , data3.proper_motions_err , data3.proper_motions_err_corr ))

In [ ]:
initial_pos = type3_sample
final_pos = type3_sample + type3_sample_pm

lines_to_plot = []

for i,_ in enumerate (initial_pos):
    longs = numpy.linspace(initial_pos[i,0], final_pos[i,0], 10)
    lats = numpy.linspace(initial_pos[i,1], final_pos[i,1], 10)
    
    line_m = GeographicToMollweide ( numpy.array ( [longs , lats] ).transpose() )
    
    lines_to_plot.append( line_m.transpose() )


In [ ]:
initial_pos_m = GeographicToMollweide ( initial_pos )
final_pos_m = GeographicToMollweide ( final_pos )
diff_m = final_pos_m - initial_pos_m

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(20,10))

t = numpy.linspace (0, 2 * numpy.pi, 100)
plt.plot ( 2 * numpy.sqrt(2) * numpy.cos(t) , numpy.sqrt(2) * numpy.sin(t) , linewidth=0.5 )

for i,_ in enumerate(initial_pos_m):
    plt.arrow( initial_pos_m[i,0] , initial_pos_m[i,1] , diff_m[i,0] , diff_m[i,1] , head_width=0.025, head_length=0.05)

plt.show()

In [ ]:
import spherepy as sp
c = sp.random_coefs(4, 4) # generate some random coefficients
print (type(c))
# sp.pretty_coefs(c)
# p = sp.ispht(c, 50, 50) # inverse spherical transform to pattern
# sp.plot_sphere_mag(p) # plot the magnitude of the pattern

In [41]:
data = import_Gaia_data("data/type2.csv")

def generate_scalar_bg (data):
    scale = 1.0e-15
    
    vsh_E_coeffs = [[0j, 1.0 * scale + 0j, 0j], [0j, 0j, 0j, 0j, 0j]]
    vsh_B_coeffs = [[0j, 0j, 0j], [0j, 0j, 0j, 0j, 0j]]
    
    model_pm = generate_model ( vsh_E_coeffs , vsh_B_coeffs , data.positions )
    
    data.proper_motions = model_pm

    data.proper_motions_err = scale * numpy.ones(data.proper_motions_err.shape, dtype=None, order='C')
    data.proper_motions_err_corr = numpy.zeros(data.proper_motions_err_corr.shape, dtype=None, order='C')
    
    return data

data = generate_scalar_bg (data)

array([[0.],
       [0.],
       [0.],
       ...,
       [0.],
       [0.],
       [0.]])

In [40]:
data = import_Gaia_data("data/type2.csv")

def generate_gr_bg (data):
    scale = 1.0e-15

    variance = numpy.array([ 0.0 , 0.3490658503988659 , 0.03490658503988659 , 0.006981317007977318 , 0.0019946620022792336 ])

    vsh_E_coeffs = [ [ scale * (numpy.random.normal(0.0 , numpy.sqrt(variance[l-1])) + (1j) * numpy.random.normal(0.0 , numpy.sqrt(variance[l-1]))) for m in range (-l,l+1)] for l in range (1,len(variance)+1)]

    for l , l_coeffs in enumerate(vsh_E_coeffs):
        if l > 0:
            for m in range (-l,l+1):
                if m < 0:
                    l_coeffs[m+l] = ((-1)**(-m)) * numpy.conj (l_coeffs[-m+l])
                elif m == 0:
                    l_coeffs[l] = numpy.real(l_coeffs[l]) + (1j) * 0.0
                
    vsh_B_coeffs = [ [ scale * (numpy.random.normal(0.0 , numpy.sqrt(variance[l-1])) + (1j) * numpy.random.normal(0.0 , numpy.sqrt(variance[l-1]))) for m in range (-l,l+1)] for l in range (1,len(variance)+1)]

    for l , l_coeffs in enumerate(vsh_B_coeffs):
        if l > 0:
            for m in range (-l,l+1):
                if m < 0:
                    l_coeffs[m+l] = ((-1)**(-m)) * numpy.conj (l_coeffs[-m+l])
                elif m == 0:
                    l_coeffs[l] = numpy.real(l_coeffs[l]) + (1j) * 0.0

    model_pm = generate_model ( vsh_E_coeffs , vsh_B_coeffs , data.positions )
    
    data.proper_motions = model_pm

    data.proper_motions_err = scale * numpy.ones(data.proper_motions_err.shape, dtype=None, order='C')
    data.proper_motions_err_corr = numpy.zeros(data.proper_motions_err_corr.shape, dtype=None, order='C')
    
    return data

data = generate_gr_bg (data)

([[0j, 0j, 0j],
  [(0.39117426637007136+0.05246644182931647j),
   (0.3849564052479421+0j),
   (-0.39117426637007136+0.05246644182931647j),
   (-0.29143302476008554+0.465867197841277j),
   (-0.23278436208940845+0.6225072674042416j)],
  [(-0.2135549821259214-0.09954648419431493j),
   (0.04950177680232255+0.2676525920743789j),
   (-0.01912944381349385+0j),
   (-0.04950177680232255+0.2676525920743789j),
   (-0.2135549821259214+0.09954648419431493j),
   (-0.03739210865090102+0.40833859706171516j),
   (-0.22858766855986293+0.06058683254654202j)],
  [(-0.06986339050675432-0.02375744420870127j),
   (0.016620751983872945+0.05024161831349102j),
   (-0.0006714160781631189-0.014726134286615442j),
   (0.07834791920721844+0j),
   (0.0006714160781631189-0.014726134286615442j),
   (0.016620751983872945-0.05024161831349102j),
   (0.06986339050675432-0.02375744420870127j),
   (-0.16348014529248847-0.09658181326786355j),
   (-0.0819806187521374+0.01268925213222059j)],
  [(-0.034306741450575384+0.07165544

In [1]:






def MollweideFunc (latitude, epsilon):
    MollweideFunc = lambda alpha : 2 * alpha + numpy.sin(2 * alpha) - numpy.pi * numpy.sin ( latitude )
        
    alpha = fsolve(MollweideFunc, latitude, xtol=epsilon)
        
    return alpha[0]
    
def GeographicToMollweide (geographicPoints):
    epsilon = 1e-6
        
    MollweidePoints = numpy.array(numpy.zeros(geographicPoints.shape))
        
    alpha = [MollweideFunc(lat, epsilon) if not numpy.isclose(numpy.absolute(lat), numpy.pi/2, rtol=epsilon, atol=0.0, equal_nan=False) else lat for lat in geographicPoints[:, 1]]
    MollweidePoints[:, 0] = (2 * numpy.sqrt(2) / numpy.pi) * ( geographicPoints[:, 0] - numpy.pi ) * numpy.cos(alpha)
    MollweidePoints[:, 1] = numpy.sqrt(2) * numpy.sin(alpha)
        
    return MollweidePoints
    
def tangent_geographic_to_Cartesian (points, dpoints):
    initial_vectors = numpy.array(points, dtype='float128', copy=True, order='K', subok=False, ndmin=0)
    dpoints = numpy.array(dpoints, dtype='float128', copy=True, order='K', subok=False, ndmin=0)
    
    end_vectors = initial_vectors + dpoints

    initial_vectors = geographic_to_Cartesian (initial_vectors)
    end_vectors = geographic_to_Cartesian (end_vectors)

    t = numpy.reciprocal ( numpy.einsum ( 'ij,ij->i' , initial_vectors , end_vectors ) )
    tangent_vectors = numpy.einsum ( 'ij,i->ij' , end_vectors , t ) - initial_vectors
    
    return tangent_vectors

def tangent_geographic_to_Cartesian2 (points, dpoints):
    tangent_vector = numpy.zeros ( ( len(points) , 3 ) , dtype = float)
    
    theta = numpy.pi / 2 - points[... , 1]
    phi = points[... , 0]
    
    dtheta = - dpoints[... , 1]
    dphi = dpoints[... , 0]
    
    tangent_vector[...,0] = numpy.cos (theta) * numpy.cos (phi) * dtheta - numpy.sin (theta) * numpy.sin (phi) * dphi
    tangent_vector[...,1] = numpy.cos (theta) * numpy.sin (phi) * dtheta + numpy.sin (theta) * numpy.cos (phi) * dphi
    tangent_vector[...,2] = - numpy.sin (theta) * dtheta
    
    return tangent_vector

def uniform_points_on_sphere_from_data (data , N):
    starting_point_ind = random.SystemRandom().randint (1 , len(data))

    points_from_data = numpy.array([starting_point_ind])
    pairwise_distances_to_data = data.dot(data[starting_point_ind].transpose())

    for i in range(1, N):
        k = pairwise_distances_to_data.argmin()

        points_from_data = numpy.append(points_from_data, k)

        new_distances_to_data = numpy.array(data.dot(data[k].transpose()))

        pairwise_distances_to_data = numpy.maximum (pairwise_distances_to_data, new_distances_to_data)
        
    return (points_from_data)

def switch_hemispheres_geographic_coords (points):
    new_points = points.copy()
    
    new_points[...,0] = numpy.where ( points[...,0] < numpy.pi , points[...,0] + numpy.pi , points[...,0] - numpy.pi )
    
    return new_points

# def simple_input_func (path):
#     points = []
    
#     with open(path) as csvfile:
#         reader = csv.reader ( csvfile )
#         next ( reader )
        
#         for row in reader:
#             ra = numpy.deg2rad ( float ( row[5] ) )
#             dec = numpy.deg2rad ( float ( row[7] ) )
            
#             points.append ( [ ra , dec ] )
    
#     points = numpy.array ( points )
#     return points

# def simple_input_func_pm (path):
#     points = []
    
#     with open(path) as csvfile:
#         reader = csv.reader ( csvfile )
#         next ( reader )
        
#         for row in reader:
#             if row[12] == "NULL":
#                 ra = numpy.nan
#             else:
#                 print(row[12])
#                 ra = numpy.deg2rad ( float ( row[12] ) )
                
#             if row[14] == "NULL":
#                 dec = numpy.nan
#             else:
#                 dec = numpy.deg2rad ( float ( row[14] ) )
            
#             points.append ( [ ra , dec ] )
    
#     points = numpy.array ( points )
#     return points